In this competition , we are asked to predict the toxicity rating of the comments taking into consideration the bias.Lets try it out.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import operator
import re
import time

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


import gc
# Any results you write to the current directory are saved as output.

import warnings
warnings.filterwarnings('ignore')

['jigsaw-unintended-bias-in-toxicity-classification', 'fasttext-crawl-300d-2m']


In [2]:
%%time

train=pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv")
test=pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
submission=pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv")

CPU times: user 12.1 s, sys: 1.66 s, total: 13.7 s
Wall time: 13.8 s


In [3]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


The column which we need to predict is the target and the comment_text contains all the comments in the forum.

In [4]:
test.head()

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


## Loading the embeddings

We use the fasttext embeddings here .All codes are borrowed/inspired from theo veils kernel -https://www.kaggle.com/theoviel/improve-your-score-with-some-text-preprocessing

In [5]:
EMB_PATH='../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'

In [6]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

def load_embed(embed_dir=EMB_PATH):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in tqdm(open(embed_dir)))
    return embedding_index

def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words


In [7]:
### Loading the embedding,

embed =load_embed()

2000001it [02:28, 13511.19it/s]


In [8]:
# vocab=build_vocab(train['comment_text'])

In [9]:
# oov_fasttext=check_coverage(vocab,embed)

We see that we dont have embeddings for close to ~10 % of the text.Lets investigate it.

In [10]:
# oov_fasttext[:10]

We see that the contractions are creating trouble.This is because the fasttext does not understand contractions.Lets clean them.

In [11]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [12]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [13]:
train['comment_text_lc']=train['comment_text'].apply(lambda x:x.lower())

In [14]:
train['comment_text_lc'].head()

0    this is so cool. it's like, 'would you want yo...
1    thank you!! this would make my life a lot less...
2    this is such an urgent design problem; kudos t...
3    is this something i'll be able to install on m...
4                 haha you guys are a bunch of losers.
Name: comment_text_lc, dtype: object

In [15]:
%%time

train['comment_text_lc']=train['comment_text_lc'].apply(lambda x:clean_contractions(x,contraction_mapping))

CPU times: user 21.9 s, sys: 360 ms, total: 22.3 s
Wall time: 22.3 s


In [16]:
# vocab=build_vocab(train['comment_text_lc'])

In [17]:
# oov_fasttext=check_coverage(vocab,embed)

There is only a marginal improvement in the embeddings list.

In [18]:
# oov_fasttext[:20]

There are also some special characters due to which we could not embed.Lets deal with them.

In [19]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [20]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [21]:
def clean_special_charac(text,punc,mapping):
    for p in mapping:
        text=text.replace(p,mapping[p])
    for p in punc:
        text=text.replace(p,f' {p} ')
        
    return(text)

In [22]:
%%time

train['comment_treated']=train['comment_text_lc'].apply(lambda x:clean_special_charac(x,punct,punct_mapping))

CPU times: user 54.6 s, sys: 0 ns, total: 54.6 s
Wall time: 54.7 s


In [23]:
train[['comment_text_lc','comment_treated']].head(10)

,comment_text_lc,comment_treated
0,"this is so cool. it is like, 'would you want y...","this is so cool . it is like , ' would yo..."
1,thank you!! this would make my life a lot less...,thank you ! ! this would make my life a lot ...
2,this is such an urgent design problem; kudos t...,this is such an urgent design problem ; kudos...
3,is this something i will be able to install on...,is this something i will be able to install on...
4,haha you guys are a bunch of losers.,haha you guys are a bunch of losers .
5,ur a sh*tty comment.,ur a sh * tty comment .
6,hahahahahahahahhha suck it.,hahahahahahahahhha suck it .
7,ffffuuuuuuuuuuuuuuu,ffffuuuuuuuuuuuuuuu
8,the ranchers seem motivated by mostly by greed...,the ranchers seem motivated by mostly by greed...
9,it was a great show. not a combo i would of ex...,it was a great show . not a combo i would of ...


In [24]:
# vocab=build_vocab(train['comment_treated'])

In [25]:
# oov_fasttext=check_coverage(vocab,embed)

Thus we were able to get close to 99.5 % of the text for embedding.

In [26]:
# oov_fasttext[:10]

Lets complete the preprocessing step for test dataset aswell

In [27]:
%%time

test['comment_text_lc']=test['comment_text'].apply(lambda x:x.lower())
test['comment_text_lc']=test['comment_text_lc'].apply(lambda x:clean_contractions(x,contraction_mapping))
test['comment_treated']=test['comment_text_lc'].apply(lambda x:clean_special_charac(x,punct,punct_mapping))

CPU times: user 4 s, sys: 4 ms, total: 4 s
Wall time: 4 s


In [28]:
test.to_csv("test_cleaned.csv",index=False)
train.to_csv("train_cleaned.csv",index=False)

In [29]:
# vocab_test=build_vocab(test['comment_treated'])
# oof_fasttext_test=check_coverage(vocab_test,embed)

In [30]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
        
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [31]:
train=reduce_mem_usage(train)

Memory usage of dataframe is 647.19 MB
Memory usage after optimization is: 551.79 MB
Decreased by 14.7%


In [32]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn import metrics
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

Using TensorFlow backend.


In [33]:
def build_matrix(word_index,embedding_index):
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

In [34]:
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
for col in identity_columns + ['target']:
    print(f'\n Converting {col} to boolean')
    train[col]=np.where(train[col]>=0.5,True,False)





 Converting male to boolean

 Converting female to boolean

 Converting homosexual_gay_or_lesbian to boolean

 Converting christian to boolean

 Converting jewish to boolean

 Converting muslim to boolean

 Converting black to boolean

 Converting white to boolean

 Converting psychiatric_or_mental_illness to boolean

 Converting target to boolean


In [35]:
#x_train=train['comment_treated']
y_train=np.where(train['target']>=0.5,True,False)*1
#y_aux_train = train[['target', 'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish','muslim', 'black', 'white', 'psychiatric_or_mental_illness']]
#x_test=test['comment_treated']


In [36]:
%%time

tokenizer = text.Tokenizer(num_words=90000)
tokenizer.fit_on_texts(list(train['comment_treated'])+list(test['comment_treated']))


CPU times: user 1min 38s, sys: 616 ms, total: 1min 38s
Wall time: 1min 38s


In [37]:
MAX_LEN=220
LSTM_UNITS=128
NUM_MODELS = 1
BATCH_SIZE = 512
DENSE_HIDDEN_UNITS = 2 * LSTM_UNITS
EPOCHS = 5

In [38]:
embedding_matrix=build_matrix(tokenizer.word_index,embed)

In [39]:
### Defining the metrics:
### https://www.kaggle.com/dborkan/benchmark-kernel/
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)




In [40]:
def calculate_overall_auc(df, model_name):
    true_labels = df['target']
    predicted_labels = df[oof_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    


In [41]:
def build_model(x_train,y_train,x_valid,y_valid,embedding_matrix,patience=3):
    
    early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
    words = Input(shape=(MAX_LEN,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = GlobalMaxPooling1D()(x)
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='tanh')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    model = Model(inputs=words, outputs=result)
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])
    model.fit(x_train,y_train,batch_size=128,epochs=3, validation_data=(x_valid, y_valid), 
                        verbose=2, callbacks=[early_stop])

    return model

In [42]:
x_train,x_valid,y_train,y_valid=train_test_split(train,y_train,test_size=0.2,random_state=100)

In [43]:
test_tokenized = tokenizer.texts_to_sequences(test['comment_text'])
X_test = sequence.pad_sequences(test_tokenized, maxlen = MAX_LEN)

In [44]:
prediction=np.zeros((len(X_test),1))
print("\n Starting tokenizing")
train_tokenised=tokenizer.texts_to_sequences(x_train['comment_treated'])
valid_tokenised=tokenizer.texts_to_sequences(x_valid['comment_treated'])
print("\n Starting padding")
X_train=sequence.pad_sequences(train_tokenised,maxlen=MAX_LEN)
X_valid=sequence.pad_sequences(valid_tokenised,maxlen=MAX_LEN)
print("\n Building model")
model = build_model(X_train, y_train, X_valid, y_valid,embedding_matrix,patience=3)
gc.collect()
print(model.summary())
print("\n Validation prediction")
pred_valid = model.predict([X_valid])
#valid_df=X_valid.copy()
#valid_df['predicted_target']=pred_valid
#bias_metrics_df = compute_bias_metrics_for_model(pred_valid, identity_columns, oof_name, 'target')
#scores.append(get_final_metric(bias_metrics_df, calculate_overall_auc(pred_valid, oof_name)))
print("\n Test set prediction")
prediction += model.predict(X_test, batch_size = 1024, verbose = 1)


 Starting tokenizing

 Starting padding

 Building model
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1443899 samples, validate on 360975 samples
Epoch 1/3
 - 463s - loss: 0.1400 - acc: 0.9458 - val_loss: 0.1285 - val_acc: 0.9480
Epoch 2/3
 - 457s - loss: 0.1260 - acc: 0.9500 - val_loss: 0.1205 - val_acc: 0.9515
Epoch 3/3
 - 458s - loss: 0.1213 - acc: 0.9516 - val_loss: 0.1241 - val_acc: 0.9497
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 220)          0                                            
___________________________

In [45]:
# # valid_checkpoint_predictions = []
# # checkpoint_predictions=[]
# # valid_weights = []
# # weights=[]
# # for model_idx in range(NUM_MODELS):
# #     model = build_model(embedding_matrix, y_aux_train_df.shape[-1])
# #     for global_epoch in range(EPOCHS):
# #         model.fit(
# #             x_train_df,
# #             [y_train_df,y_aux_train_df],
# #             batch_size=BATCH_SIZE,
# #             epochs=1,
# #             verbose=2,
# #             callbacks=[
# #                 LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))
# #             ]
# #         )
# #         valid_checkpoint_predictions.append(model.predict(x_valid_df, batch_size=2048)[0].flatten())
# #         valid_weights.append(2 ** global_epoch)
# # checkpoint_predictions.append(model.predict(x_test,batch=2048)[0].flatten())


# # predictions = np.average(checkpoint_predictions, axis=0)

# def train_model(X, X_test, y):
    
#     oof = np.zeros((len(X), 1))
#     prediction = np.zeros((len(X_test), 1))
#     scores = []
#     print("\n Started tokenizing for test set")
#     test_tokenized = tokenizer.texts_to_sequences(test['comment_text'])
#     X_test = sequence.pad_sequences(test_tokenized, maxlen = MAX_LEN)
#     print("\n Completed tokenizing and padding.Starting the train-valid epochs.")
#     for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
#         print('Fold', fold_n, 'started at', time.ctime())
#         X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
#         y_train, y_valid = y[train_index], y[valid_index]
#         valid_df = X_valid.copy()    
#         print(f'\n Tokenizing for fold {fold_n}')
#         train_tokenised=tokenizer.texts_to_sequences(X_train['comment_treated'])
#         valid_tokenised=tokenizer.texts_to_sequences(X_valid['comment_treated'])
#         print(f'\n Tokenising completed.Starting padding for fold {fold_n}')
#         X_train=sequence.pad_sequences(train_tokenised,maxlen=MAX_LEN)
#         X_valid=sequence.pad_sequences(valid_tokenised,maxlen=MAX_LEN)
#         print(f'\n Padding completed .Started model building for fold {fold_n}')
#         model = build_model(X_train, y_train, X_valid, y_valid,embedding_matrix,patience=3)
#         print(f'\n Model building completed for fold {fold_n}')
#         pred_valid = model.predict(X_valid)
#         oof[valid_index] = pred_valid
#         valid_df[oof_name] = pred_valid
#         print("Started calculating the bias metric and final metric")
#         bias_metrics_df = compute_bias_metrics_for_model(valid_df, identity_columns, oof_name, 'target')
#         scores.append(get_final_metric(bias_metrics_df, calculate_overall_auc(valid_df, oof_name)))
#         print("Completed finding the bias metric.Started prediction for test set")
#         prediction += model.predict(X_test, batch_size = 1024, verbose = 1)
    
#     prediction /= n_fold
    
#     # print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
#     return oof, prediction, scores

In [46]:
# oof_name = 'predicted_target'
# oof, prediction, scores = train_model(X=train, X_test=test, y=y_train)
# print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))

In [47]:
submission['prediction']=prediction

In [48]:
submission.to_csv('submission.csv', index=False)

In [49]:
submission.head()

,id,prediction
0,7000000,0.001145
1,7000001,0.000225
2,7000002,0.033325
3,7000003,0.000987
4,7000004,0.994221
